## CHPATER 7 합성곱 신경망

- CNN은 이미지 인식과 음성 인식 등 다양한 곳에서 사용되는데, 특히 이미지 인식 분야에서 딥러닝을 활용한 기법은 거의 다 CNN을 기초로 한다.

## 7.1 전체 구조

- CNN도 지금까지 본 신경망과 같이 레고 블록처럼 계층을 조합하여 만들 수 있다. 다만 합성곱 계층(Convolutional layer)과 풀링 계층(Pooling layer) 새롭게 등장한다.
- 지금까지 본 신경망은 인접하는 계층의 모든 뉴런과 결합되어 있었다. 이를 완전연결(fully-connected)라 하며, 완전히 연결된 계층을 Affine 계층이라는 이름으로 구현했다.

![test](./img/합성곱.png)

- 첫번째 사진은 완전연결 계층(Affine 계층)으로 이뤄진 네트워크의 예
> - Affine 계층 뒤에 활성화 함수를 갖는 ReLu계층이 이어지고 Affine-ReLu 조합이 4개가 쌓였고, 마지막 5번째 계층은 Affine 계층에 이어 소프트맥스에서 최종 결과(확률)을 출력한다.
- 두번째 사진은 CNN으로 이뤄진 네트워크의 예
> - CNN의 계층은 Affine - ReLu - (Pooling) 흐름으로 연결된다. (풀링 계층은 생략되기도 한다.)
> - CNN에서 또 다른 점은 출력에 가까운 층에서는 지금까지의 'Affine-ReLu' 구성을 사용할 수 있다는 점이다. 또 마지막 출력층에서는 'Affine-Softmax' 조합을 그대로 사용한다.

## 7.2 합성곱 계층

- CNN에서는 패딩(padding), 스트라이드(stride) 등 CNN의 고유의 용어가 등장한다.
- 또 각 계층 사이에는 3차원 데이터 같이 입체적인 데이터가 흐른다는 점에서 완전연결 신경망과 다르다.

## 7.2.1 완전연결 계층의 문제점

- 완전연결 계층에서는 인접하는 계층의 뉴런이 모두 연결되고 출력의 수는 임의로 정할 수 있다.
- 하지만 데이터의 형상이 무시 된다는 사실이다. 
> - 입력 데이터가 이미지인 경우를 예로 들면, 이미지는 통상 세로,가로,채널(색상)로 구성된 3차원 데이터이다. 
> - 그러나 완전연결 계층에 입력할 때는 3차원 데이터를 평평한 1차원 데이터로 평탄화해줘야 한다.
>> - 즉, 3차원 데이터를 평평한 1차원 데이터로 평탄화하여 이미지를 1줄로 세운 784개의 데이터를 Affine 계층에 입력한 것이다.
> - 이미지는 3차원 형상이며, 이 형상에는 소중한 공간적 정보가 담겨 있다. 
>> - 예를 들어 공간적으로 가까운 픽셀은 값이 비슷하거나, RGB의 각 채널은 서로 밀접하게 관련되어 있거나, 거리가 먼 픽셀끼리는 별 연관이 없는 등, 3차원 속에서 의미를 갖는 본질적인 패턴이 숨어 있을 것이다. 그러나 완전연결 계층은 형상을 무시하고 모든 입력 데이터를 동등한 뉴런으로 취급하여 형상이 담긴 정보를 살릴 수 없다.
- 한편, 합성곱 계층은 형상을 유지한다. 이미지도 3차원 데이터로 입력받으며, 마찬가지로 다음 계층에도 3차원 데이터로 전달한다. 그래서 CNN에서는 이미지처럼 형상을 가진 데이터를 제대로 이해할(가능성이 있는) 것이다.
- CNN에서는 합성곱 계층의 입출력 데이터를 특징 맵(feature map)이라고도 합니다. 
> - 합성곱 계층의 입력 데이터를 입력 특징 맵(input feature map), 출력 데이터를 출렵 특징 맵(output feature map)이라고 하는 식이다.

## 7.2.2 합성곱 연산

- 합성곱 계층에서의 합성곱 연산을 처리한다. 
- 합성곱 연산은 이미지 처리에서 말하는 필터 연산에 해당한다.

![test](./img/합성곱연산.png)

- 위 그림과 같이 합성곱 연산은 입력 데이터에 필터를 적용한다. 데이터 필터의 형상을 (높이height,너비width)로 표기하며, 이 예에서는 입력은 (4,4), 필터는 (3,3), 출력은 (2,2)가 된다. 문헌에 따라 필터를 커널이라 칭하기도 한다.
- 합성곱 연산은 필터의 윈도우(window)를 일정 간격으로 이동해가며 입력 데이터에 적용한다.
> - 즉, 입력과 필터에서 대응하는 원소끼리 곱한 후 그 총합을 구한다.
- 완전연결 신경망에는 가중치 매개변수와 편향이 존재했는데, CNN에서는 필터의 매개변수가 그동안의 가중치에 해당한다. 그리고 CNN에도 편향이 존재하는데 위 그림은 필터를 적용하는 단계까지만 보여준 것이고, 편향까지 포함하면 아래와 같은 흐름이 된다.

![test](./img/편편향.png)

- 그리고 편향은 항상 하나(1x1)만 존재한다. 그 하나의 값을 필터를 적용한 모든 원소에 더하는 것이다.

## 7.2.3 패딩

- 합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값(예컨데 0)으로 채우기도 한다. 이를 패딩이라하며 합성곱 연산에서 자주 이용하는 기법이다.

![test](./img/패딩.png)

- 위 그림은 (4,4) 크기의 입력 데이터에 폭이 1인 패딩을 적용한 모습이다.
> - 폭 1짜리 패딩이라 하면 입력 데이터 사방 1픽셀을 특정 값으로 채우는 것이다.
- 처음 크기가 (4,4)인 입력 데이터에 패딩이 추가되어 (6,6)이 된다. 이 입력에 (3,3)크기의 필터를 걸면 (4,4) 크기의 출력 데이터가 생성된다. 
- 패딩은 주로 출력 크기를 조정할 목적으로 사용한다. 예를 들어 (4,4) 입력 데이터에 (3,3) 필터를 적용하면 출력은 (2,2)가 되어, 입력보다 2만큼 줄어든다. 이는 합성곱 연산을 몇 번이나 됲ㄹ이하는 심층 신경망에서는 문제가 될 수 있다. 
- 즉, 합성곱 연산을 거칠 때마다 크기가 작아지면 어느 시점에서는 출력의 크기가 1이 되어버리기 때문에 더 이상 연산을 적용할 수 없다는 문제점이 생긴다.
- 이러한 사태를 막기 위해 패딩을 사용한다.

## 7.2.4 스트라이드

- 필터를 적용하는 위치의 간격을 스트라이드(stride)라 한다.
- 스트라이드가 2인 합성곱 연산

![test](./img/스트라이드.png)

- 크기가 (7,7)인 입력 데이터에 스트라이드를 2로 설정한 필터를 적용하면 (3,3)이 된다. 이처럼 스트라이드를 키우면 출력 크기는 작아진다. 한편, 패딩을 크게 하면 출력 크기가 커진다. 이러한 관계를 수식화하면

![test](./img/수식.png)

- 입력의 크기를 (H,W), 필터 크기를 (FH,FW), 출력 크기를 (OH,OW), 패딩을 P, 스트라이드를 S라 하면, 출력 크기는 위 식으로 계산된다.
- 정수로 나눠떨어지는 값이어야 한다는 점에 주의하라

## 7.2.5 3차원 데이터의 합성곱 연산

- 위 경우는 2차원 형상을 다루는 합성곱 연산이였다. 
- 이미지 같은 경우는 세로,가로,채널 까지 고려한 3차원 데이터로 다음과 같이 나타난다.

![test](./img/삼차원.png)

- 채널 쪽으로 특징 맵이 여러 개 있다면 입력 데이터와 필터의 합성곱 연산을 채널마다 수행하고, 그 결과를 더해서 하나의 출력을 얻는다.
- 3차원의 합성곱 연산에서 주의할 점은 입력 데이터의 채널 수와 필터의 채널 수가 같아야 한다는 점이다.
> - 필터 자체의 크기는 원하는 값으로 설정할 수 있다.

## 7.2.6 블록으로 생각하기

- 3차원의 합성곱 연산은 데이터와 필터를 직육면체 블록이라 생각하면 쉽다.
- 3차원 데이터를 다차원 배열로 나타낼 때는 (채널,높이,너비)순서로 쓴다.

![test](./img/블록블록.png)

- 채널 수 C, 높이 H,너비 W의 형상은 (C,H,W) 필터의 경우도 같다.

![test](./img/블록.png)

- 위 그림은 합성곱 연산의 출력으로 다수의 채널을 내보내는 즉 필터(가중치)를 다수를 사용하여 다수의 채널을 내보내고 있다.
- 필터를 FN개 적용하면 출력 맵도 FN개가 생성된다. 그리고 FN개의 맵을 모으면 형상이 (FN,OH,OW)인 블록이 완성된다. 
- 합성곱 연산에서는 필터의 수도 고려해야 한다. 그런 이유로 필터의 가중치 데이터는 4차원 데이터이며 (출력 채널수, 입력 채널 수, 높이, 너비) 순으로 쓴다. 

![test](./img/합성곱처리.png)

- 편향은 채널 하나에 값 하나씩으로 구성된다.
- 위 예에서는 편향의 형상은 (FN,1,1)이고 필터의 출력 결과의 형상은 (FN,OH,OW)이다. 이들 두 블록을 더하면 편향의 각 값이 필터의 출력인 (FN,OH,OW) 블록의 대응 채널의 원소 모두에 더해진다.

## 7.2.7 배치 처리

- 합성곱 연산도 마찬가지로 배치 처리를 지원하고자 합니다. 그래서 각 계층을 흐르는 데이터의 차원을 하나 늘려 4차원 데이터로 저장한다. 구체적으로는 데이터를 (데이터 수,채널 수, 높이, 너비) 순으로 저장한다. 데이터가 N개일 때 배치 처리는 다음과 같다.

![test](./img/합성배치.png)

- 배치 처리 시의 데이터 흐름을 나타낸 위 그림을 보면 각 데이터의 선두에 배치용 차원을 추가했다. 즉 4차원 형상을 가진 채 각 계층을 타고 흐른다.
- 신경망에서 4차원 데이터가 하나 흐를 때마다 데이터 N개에 대한 합성곱 연산이 이뤄진다는 것이다. 즉, N회 분의 처리를 한 번에 수행하는 것이다.

## 7.3 풀링 계층

- 풀링은 세로ㆍ가로 방향의 공간을 줄이는 연산이다. 

![test](./img/풀링계층.png)

- 위 그림은 2X2 최대 풀링(max pooling)을 스트라이드 2로 처리하는 순서이다. 즉, 2X2 최대풀링은 그림과 같이 2X2 크기의 영역에서 가장 큰 원소 하나를 꺼낸다. 
- 스트라이드는 이 예에서는 2로 설정했으므로 2X2 윈도우가 원소 2칸 간격으로 이동한다.
- 풀링의 윈도우 크기와 스트라이드는 같은 값으로 설정하는게 보통이다.
> - 즉, 윈도우가 3X3이면 스트라이드는 3으로, 윈도우가 4X4이면 스트라이드는 4로 설정한다.

## 7.3.1 풀링 계층의 특징

- 학습해야 할 매개변수가 없다.
> - 풀링 계층은 합성곱 계층과 달리 학습해야 할 매개변수가 없다. 즉, 풀링은 대상영역에서 최대값이나 평균을 취하는 명확한 처리이므로 특별히 학습할 것이 없다.
- 채널 수가 변하지 않는다.
> - 풀링 연산은 입력 데이터의 채널 수 그대로 출력 데이터로 내보낸다. 즉, 채널마다 독립적으로 계산
- 입력의 변화에 영향을 적게 받는다.
> - 입력 데이터가 조금 변해도 풀링의 결과는 잘 변하지 않는다.

![test](./img/풀링영향.png)

## 7.4 합성곱/풀링 계층 구현하기

## 7.4.1 4차원 배열

In [1]:
import numpy as np

In [2]:
x = np.random.rand(10,1,28,28) # 데이터:10, 채널:1, 높이:28, 너비:28

In [4]:
x.shape

(10, 1, 28, 28)

In [5]:
x[0].shape # (10개 중) 첫 번째 데이터에 접근 

(1, 28, 28)

In [6]:
x[1].shape # (10개 중) 두 번째 데이터에 접근

(1, 28, 28)

In [8]:
x[0,0].shape # 첫 번째 데이터에 첫 채널의 공간에 접근

(28, 28)

## 7.4.2 im2col로 데이터 전개하기

- 합성곱 연산을 for문을 사용하여 구현하면 복잡할 뿐더라 효율성이 떨어진다.
- im2col은 입력 데이터를 필터링(가중치 계산)하기 좋게 전개하는 (펼치는) 함수이다.

![test](./img/im2col.png)

- 위 그림과 같이 3차원 입력 데이터에 im2col을 적용하면 2차원 행렬로 바뀐다. (정확히는 배치 안의 데이터 수까지 포함한 4차원 데이터를 2차원으로 변환한다.)

- im2col은 필터링하기 좋게 입력 데이터를 전개한다.

![test](./img/im2col1.png)

- 즉, 위 그림과 같이 입력 데이터에서 필터를 적용하는 영역(3차원 블록)을 한 줄로 늘어놓는다. 이 전개를 적용하는 모든 영역에서 수행하는 게 im2col이다.
- 위 그림에서는 보기에 좋게끔 스트라이드를 크게 잡아 필터의 적용 영역이 겹치지 않도록 했지만, 실제 상황에서는 겹치는 경우가 대부분이다. 
- 필터 적용 영역이 겹치게 되면 im2col로 전개한 후의 원소 수가 원래 블록의 원소 수보다 많아진다.
- 따라서 im2col을 사용해 구현하면 메모리를 더 많이 소비하는 단점이 있다. 
- 하지만 컴퓨터는 큰 행렬을 묶어서 계산하는 데 탁월하다.
- im2col은 'image to column 즉, 이미지에서 행렬로 라는 뜻이다.'

![test](./img/im2col.png)

- 위 그림 처럼 im2col로 입력 데이터를 전개한 다음에는 합성곱 계층의 필터(가중치)를 1열로 전개하고, 두 행렬의 곱을 계산하면 된다. 즉, 완전연결 계층의 Affine계층에서 한 것과 비슷하다.
- 마지막으로 2차원 출력을 다시 4차원으로 변형하는 과정을 거친다.

## 7.4.3 합성곱 계층 구현하기

In [9]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [10]:
# im2col(input_data,filter_h,filter_w,stride,pad)
# input_data(데이터 수, 채널 수, 높이, 너비)의 4차원 배열로 이뤄진 입력 데이터
# fliter_h - 필터의 높이
# fliter_w - 필터의 너비
# stride - 스트라이드
# pad - 패딩

In [11]:
import sys, os
sys.path.append(os.pardir)

In [12]:
x1 = np.random.rand(1,3,7,7) # 데이터 수, 채널 수, 높이, 너비
col1 = im2col(x1,5,5,stride=1,pad=0)
print(col1.shape)

(9, 75)


In [13]:
x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2,5,5,stride=1,pad=0)
print(col2.shape)

(90, 75)


- 첫 번째는 배치 크기가 1(데이터1개), 채널은 3개, 높이ㆍ너비가 7X7의 데이터이고, 두번째는 배치 크기만 10이고 나머지는 첫번째와 같다.
- 두 경우 모두 2번째 차원의 원소는 75개이다. 이 값은 필터의 원소 수와 같다(채널3개,5x5 데이터) 또한 배치 크기가 1일 때는 im2col의 결과의 크기가 (9,75)이고 10일 때는 그 10배인 (90,75) 크기의 데이터가 저장된다.

In [14]:
class Convolution:
    def ___init__(self,W,b,stride=1,pad=0): 
        # 필터(가중치), 편향, 스트라이드, 패딩을 인수로 받아 초기화
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    def forward(self,x):
        FN,C,FH,FW = self.W.shape 
        # 필터는 4차원 형상 (FN: 필터 개수, C: 채널, FH: 필터 높이, FW: 필터 너비)
        N, C, H, W = x.shape
        out_h = int(1+(H+2*self.pad-FH)/self.stride)
        out_w = int(1+(W+2*self.pad-FW)/self.stride)
        
        col = im2col(x,FH,FW,self.stride,self.pad)
        col_W = self.W.reshape(FN,-1).T 
        # 필터 전개, 2차원배열로 전개
        # reshape -1을 지정하면 다차원 배열의 원소 수가 변환 후에도 똑같이 유지 되도록 적절히 묶음
        # 즉, (10,3,3,5) 형상을 한 다차원 배열 원소 수는 총 750 
        # reshape(10,-1) 을 호출하면 750개의 원소를 10묶음으로 즉, (10,75)인 배열로 만들어준다.
        out = np.dot(col,col_W) + self.b # 두 행렬의 곱
        
        out = out.reshape(N,out_h,out_w,-1).transpose(0,3,1,2)
        # 형상 (N,H,W,C) -> (N,C,H,W)
        # 인덱스 (0,1,2,3) -> (0,3,1,2)
        
        return out

In [15]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.
    
    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

## 7.4.4 풀링 계층 구현하기

- 풀링 계층 구현도 합성곱 계층과 마찬가지로 im2col을 사용해 입력 데이터를 전개한다. 
- 단, 풀링의 경우엔 채널 쪽이 독립이라는 점이 합성곱 계층 때와 다르다.
> - 구체적으로는 풀링 적용 영역을 채널마다 독립적으로 전개

![test](./img/풀링전개.png)

- 위 그림처럼 전개한 후, 전개한 행렬에서 행별 최댓값을 구하고 적절한 형상으로 성형하기만 하면 된다.

![test](./img/풀링구현.png)

In [17]:
class Pooling:
    def __init__(self,pool_h,pool_w,stride=1,pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self,x):
        N, C, H, W = x.shape
        out_h = int(1+(H-self.pool_h)/self.stride)
        out_w = int(1+(W-self.pool_w)/self.stride)
        
        col = im2col(x,self.pool_h,self.pool_w,self.stride,self.pad)
        col = col.reshape(-1,self.pool_h*self.pool_w)
        
        out = np.max(col,axis=1)
        
        out = out.reshape(N,out_h,out_w,C).transpose(0,3,1,2)
        
        return out
    
    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx
# 1. 입력 데이터를 전개한다.
# 2. 행별 최댓값을 구한다.
# 3. 적절한 모양으로 성형한다.

## 7.5 CNN 구현하기

![test](./img/합성곱.png)

In [19]:
import sys, os
sys.path.append(os.pardir)
import pickle
import numpy as np
from collections import OrderedDict

class SimpleConvNet:
	def __init__(self, input_dim=(1, 28, 28),
				 conv_param={'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
				 hidden_size=100, output_size=10, weight_init_std=0.01):
		filter_num = conv_param['filter_num']
		filter_size = conv_param['filter_size']
		filter_pad = conv_param['pad']
		filter_stride = conv_param['stride']
		input_size = input_dim[1]
		conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
		pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
		
		self.params = {
			'W1': weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size),
			'b1': np.zeros(filter_num),
			'W2': weight_init_std * np.random.randn(pool_output_size, hidden_size),
			'b2': np.zeros(hidden_size),
			'W3': weight_init_std * np.random.randn(hidden_size, output_size),
			'b3': np.zeros(output_size),
		}
		
		self.layers = OrderedDict()
		self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
										   conv_param['stride'], conv_param['pad'])
		self.layers['Relu1'] = Relu()
		self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
		self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
		self.layers['Relu2'] = Relu()
		self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
		
		self.last_layer = SoftmaxWithLoss()
		
	def predict(self, x):
		for layer in self.layers.values():
			x = layer.forward(x)
			
		return x
	
	def loss(self, x, t):
		y = self.predict(x)
		return self.last_layer.forward(y, t)
	
	def accuracy(self, x, t, batch_size=100):
		if t.ndim != 1:
			t = np.argmax(t, axis=1)
			
		acc = 0.0
		
		for i in range(int(x.shape[0] / batch_size)):
			tx = x[i*batch_size:(i+1)*batch_size]
			tt = t[i*batch_size:(i+1)*batch_size]
			y = self.predict(tx)
			y = np.argmax(y, axis=1)
			acc += np.sum(y == tt)
			
		return acc / x.shape[0]
	
	def numerical_gradient(self, x, t):
		loss_w = lambda w: self.loss(x, t)
		
		grads = {}
		for idx in (1, 2, 3):
			grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
			grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])
			
		return grads

	def gradient(self, x, t):
		self.loss(x, t)

		dout = 1
		dout = self.last_layer.backward(dout)

		layers = list(self.layers.values())
		layers.reverse()
		for layer in layers:
			dout = layer.backward(dout)

		grads = {
			'W1': self.layers['Conv1'].dW,
			'b1': self.layers['Conv1'].db,
			'W2': self.layers['Affine1'].dW,
			'b2': self.layers['Affine1'].db,
			'W3': self.layers['Affine2'].dW,
			'b3': self.layers['Affine2'].db,
		}

		return grads

	def save_params(self, file_name='params.pkl'):
		params = {}
		for key, val in self.params.items():
			params[key] = val
		with open(file_name, 'wb') as f:
			pickle.dump(params, f)

	def load_params(self, file_name='params.pkl'):
		with open(file_name, 'rb') as f:
			params = pickle.load(f)
		for key, val in params.items():
			self.params[key] = val

		for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
			self.layers[key].W = self.params['W' + str(i+1)]
			self.layers[key].b = self.params['b' + str(i+1)]


## 7.6 CNN 시각화하기

- 합성곱 계층을 시각화해서 CNN이 보고 있는 것이 무엇인지 알아본다.

## 7.6.1 1번째 층의 가중치 시각화하기

- 1번째 층의 합성곱 계층의 가중치는 그 형상이 (30,1,5,5)였다.(필터30개,채널1개,5X5크기). 필터의 크기가 5X5이고 채널이 1개라는 것은 이 필터를 1채널의 회색조 이미지로 시각화 할 수 있다.

- 학습전
![test](./img/학습전.png)

- 학습후
![test](./img/학습후.png)

- 학습 전 필터는 무작위로 초기화되고 있어 흑백의 정도에 규칙성이 없다. 한편, 학습을 마친 필터는 규칙성이 있는 이미지가 되었다.
> - 흰색에서 검은색으로 점차 변화하는 필터와 덩어리가 진 필터 등, 규칙을 띄는 필터로 바뀌었다.
- 학습 후 규칙성이 있는 필터는 에지(색상이 바뀐 경계선)와 블롭(국소적으로 덩어리진 영역)등을 보고 있다. 

![test](./img/에지.png)

- 가령 왼쪽 절반이 흰색이고 오른쪽 절반이 검은색인 필터와 같이 세로 방향의 에지에 반응하는 필터이다.(필터 1,2 참조)
- 이처럼 합성곱 계층의 필터는 에지나 블롭 등의 원시적인 정보를 추출할 수 있다.

## 7.6.2 층 깊이에 따른 추출 정보 변화

- 겹겹이 쌓인 CNN은 계층이 깊어질수록 추출되는 정보(정확히는 강하게 반응하는 뉴런)는 더 추상화 된다는 것을 알 수 있다.

![test](./img/알렉스넷.png)

- 위 그림은 합성곱 계층과 풀링 계층을 여러 겹 쌓고, 마지막으로 완전연결 계층을 거쳐 결과를 출력하는 구조이다.
- 1번째 층은 에지와 블롭, 3번째 층은 텍스쳐, 5번째 층은 사물의 일부, 마지막 완전연결 계층은 사물에 뉴런이 반응한다.
- 즉, 층이 깊어지면서 뉴런이 반응하는 대상이 단순한 모양에서 '고급'정보로 변화해간다. 다시 말하면 사물의 의미를 이해하도록 변화한다.

## 7.7 대표적인 CNN
- CNN의 원조인 LeNet이고, 다른 하나는 딥러닝이 주목받도록 이끈 AlexNet이다

# 7.7.1 LeNet

- LeNet은 손글씨 숫자를 인식하는 네트워크로, 1988년에 제안

![test](./img/LeNet.png)

- 위 그림과 같이 합성곱 계층과 풀링 계층(정확히는 단순히 원소를 줄이기만 하는 서브샘플링 계층)을 반복하고, 마지막으로 완전연결 계층을 거치면서 결과를 출력
- LeNet과 현재의 CNN의 차이점
> - 활성화함수이다. LeNet은 시그모이드 함수를 사용하는 반면에 현재는 주로 ReLu를 사용한다.
> - 원래의 LeNet은 서브샘플링을 하여 중간 데이터의 크기가 작아지지만 현재는 최대 풀링이 주류이다.

## 7.7.2 AlexNet

- 2012년에 발표된 AlexNet은 딥러닝 열풍을 일으키는 데 큰 역할을 했다.

![test](./img/AlexNet.png)

- AlexNet은 합성곱 계층과 풀링 계층을 거듭하며 마지막으로 완전연결 계층을 거쳐 결과를 출력한다. 
- LeNet과 큰 구조는 바뀌지 않았지만 다음과 같은 차이점이 있다.
> - 활성화 함수로 ReLu함수를 사용
> - LRN(local response normalization) 이라는 국소적 정규화를 실시하는 계층을 이용
> - 드롭아웃을 사용한다.
- LeNet과 AlexNet의 큰 차이는 없지만 이를 둘러싼 환경과 컴퓨터 기술이 큰 진보를 이룬 것이다.
- 대량의 데이터를 누구나 얻을 수 있게 되었고, 병렬 계산에 특화된 GPU가 보급되면서 대량의 연산을 고속으로 수행할 수 있게 되었다. 빅데이터와 GPU, 이것이 딥러닝 발전의 큰 원동력이다.

## 7.8 정리

- CNN은 지금까지의 완전연결 계층 네트워크에 합성곱 계층과 풀링 계층을 새로 추가한다.
- 합성곱 계층과 풀링 계층은 im2col을 이용하면 간단하고 효율적으로 구현
- CNN을 시각화해보면 계층이 깊어질수록 고급 정보가 추출되는 모습을 확인 할 수 있다.